# 1. Email Uplift Analysis: Regression Discontinuity + ML

Goal:
- Measure whether marketing emails increase near-term movie ticket purchases.

Approach:
- Build an around-the-day (ADT) panel for +/- K days around each email to enable an RDD-style before/after contrast.
- Train an ML model (CatBoost) to capture non-linearities and interactions, and assess the post-email effect.
- Fit a GLM with cluster-robust SE at customer level to estimate the post-email discontinuity.

Deliverables:
- Clear uplift estimate with uncertainty.
- Feature insights from ML (SHAP) to understand drivers.
- Actionable recommendations and caveats for stakeholders.

In [ ]:
# Imports: core DS stack and modeling libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid

# Stats and modeling
import statsmodels.api as sm
import statsmodels.formula.api as smf
import holidays
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool
import shap

# Reproducibility
np.random.seed(42)

# Plot style
sns.set(style="whitegrid", context="talk")

---
# 2. Data

We load transactions, pricebook, customers, emails, and releases. Dates are parsed and basic integrity filters are applied.

Insights (summary):
- We keep only valid transactions with positive tickets and revenue.
- Email timestamps are normalized to date granularity for daily paneling.
- Releases are filtered to those with positive sales.

In [ ]:
# Read raw data CSVs from the repo data/ folder
# Note: paths are relative to this notebook location (src/)
df_transactions = pd.read_csv("../data/Transactions.csv", header=0)
df_pricebook = pd.read_csv("../data/Pricebook.csv", header=0)
df_customers = pd.read_csv("../data/Customers.csv", header=0)
df_emails = pd.read_csv("../data/Emails.csv", header=0)
df_releases = pd.read_csv("../data/Releases.csv", header=0, sep="|")

print("Shapes:")
print("Transactions:", df_transactions.shape)
print("Pricebook:", df_pricebook.shape)
print("Customers:", df_customers.shape)
print("Emails:", df_emails.shape)
print("Releases:", df_releases.shape)

In [ ]:
# Transactions wrangling
# - Create a transaction_id per (member, transaction date, show date)
# - Keep only positive tickets and revenue
# - Aggregate duplicates per ID; parse dates

df_transactions['transaction_id'] = df_transactions.apply(
    lambda x: str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{x['CARD_MEMBERSHIPID']}_{x['FECHA_TRANSACCION']}_{x['FECHA_FUNCION']}")),
    axis=1
)

# Filter for valid sales
df_transactions = df_transactions.query('BOLETOS > 0').reset_index(drop=True)
df_transactions = df_transactions.query('IMPORTE_TAQUILLA > 0').reset_index(drop=True)

# Aggregate per transaction_id
cols_agg = {
    'ID_CINE': 'first',
    'FECHA_TRANSACCION': 'first',
    'FECHA_FUNCION': 'first',
    'CARD_MEMBERSHIPID': 'first',
    'ID_MARCA': 'first',
    'TX_PELICULA_UNICA': 'first',
    'BOLETOS': 'sum',
    'IMPORTE_TAQUILLA': 'sum'
}

df_transactions = (
    df_transactions
    .groupby('transaction_id', as_index=False)
    .agg(cols_agg)
    .assign(
        FECHA_TRANSACCION=lambda x: pd.to_datetime(x['FECHA_TRANSACCION'], format='%Y-%m-%d', errors='coerce'),
        FECHA_FUNCION=lambda x: pd.to_datetime(x['FECHA_FUNCION'], format='%Y-%m-%d', errors='coerce')
    )
    .sort_values('FECHA_TRANSACCION', ascending=True, ignore_index=True)
)

print("Transactions after cleaning:", df_transactions.shape)
df_transactions.head()

In [ ]:
# Join emails with customers for a consistent subscriber set
# - Normalize timestamps to dates and extract hour for potential use

df_emails = (
    df_emails
    .merge(df_customers, how='inner', left_on="SubscriberKeyH", right_on="SubscriberKey")
    .drop(columns=["SubscriberKeyH", "SubscriberKey"])
    .reset_index(drop=True)
)

# Parse timestamp and derive date and hour
df_emails['EventDate_TZ'] = pd.to_datetime(df_emails['EventDate'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')
df_emails['EventDate'] = df_emails['EventDate_TZ'].dt.date

df_emails['EventDate_Hour'] = df_emails['EventDate_TZ'].dt.hour

print("Emails post-merge:", df_emails.shape)

df_releases['ESTRENO'] = pd.to_datetime(df_releases['ESTRENO'], format='%Y-%m-%d', errors='coerce')
df_releases = df_releases.query('VENTAS > 0').reset_index(drop=True)

print("Releases after basic cleaning:", df_releases.shape)

In [ ]:
# Build a holiday calendar for Mexico over the transactions coverage
start_date = df_transactions['FECHA_TRANSACCION'].min()
end_date = df_transactions['FECHA_TRANSACCION'].max()
date_range = pd.date_range(start=start_date, end=end_date)
mx_holidays = holidays.MX(years=date_range.year.unique())

holiday_dates = [d for d in date_range if d in mx_holidays]
holiday_names = [mx_holidays[d] for d in holiday_dates]
df_mx_holidays = pd.DataFrame({'date': holiday_dates, 'holiday': holiday_names})

df_mx_holidays.head()

In [ ]:
# Compute per-movie windows and a blockbuster intensity index over time
# - For each movie: from 2 days before release to last show in transactions
# - Blockbuster score: log2 of sales (VENTAS); also a smoothed time series

df_releases_range = (
    df_transactions
    .groupby('TX_PELICULA_UNICA', as_index=False)
    .agg(
        date_end=('FECHA_FUNCION', 'max'),
        tickets_sold=('BOLETOS', 'sum'),
        money_sold=('IMPORTE_TAQUILLA', 'sum'),
    )
    .merge(df_releases, how='inner', on='TX_PELICULA_UNICA')
    .assign(
        days_on_air=lambda x: (x['date_end'] - x['ESTRENO']).dt.days,
        blockbuster_score=lambda x: np.log2(x['VENTAS']),
        date_before_release=lambda x: x['ESTRENO'] - pd.Timedelta(days=2),
    )
)

# Build daily series of concurrent movies and total blockbuster score
date_range = pd.date_range(start=df_releases_range['date_before_release'].min(),
                           end=df_releases_range['date_end'].max())

ts_movies_on_air = (
    pd.DataFrame({'date': date_range})
    .assign(
        on_air=lambda x: x['date'].apply(
            lambda d: df_releases_range.query('date_before_release.le(@d) and date_end.ge(@d)').shape[0]
        ),
        blockbuster_score=lambda x: x['date'].apply(
            lambda d: df_releases_range.query('date_before_release.le(@d) and date_end.ge(@d)')['blockbuster_score'].sum()
        )
    )
)

# look
ts_movies_on_air.head()

In [ ]:
# Infer child presence from ticket unit price vs. reference price
# - Join pricebook by cinema; if unit price < reference, mark as child ticket
# - Build per-customer flags and aggregates (has_children)

df_transactions = df_transactions.merge(df_pricebook, how='left', on='ID_CINE')

df_transactions = (
    df_transactions
    .assign(
        had_children_ticket=lambda x: np.where(
            (x['IMPORTE_TAQUILLA'] / x['BOLETOS']).lt(x['PRECIO_POL_R']), 1, 0
        ),
        has_children=lambda x: x.groupby('CARD_MEMBERSHIPID')['had_children_ticket'].transform(lambda y: y.any().astype(int))
    )
)

print("Child-ticket transactions:", int(df_transactions['had_children_ticket'].sum()))
print("% child-ticket transactions:", df_transactions['had_children_ticket'].mean() * 100)

# Build time-series aggregates per customer (visits, spend)
df_transactions = (
    df_transactions
    .sort_values(by=['CARD_MEMBERSHIPID', 'FECHA_TRANSACCION'], ignore_index=True)
    .assign(
        cum_visits=lambda x: x.groupby('CARD_MEMBERSHIPID')['transaction_id'].cumcount() + 1,
        cum_spend=lambda x: x.groupby('CARD_MEMBERSHIPID')['IMPORTE_TAQUILLA'].cumsum(),
        avg_spend_per_visit=lambda x: x.groupby('CARD_MEMBERSHIPID')['IMPORTE_TAQUILLA'].transform('mean'),
        total_visits=lambda x: x.groupby('CARD_MEMBERSHIPID')['transaction_id'].transform('count'),
    )
    .sort_values(['total_visits', 'CARD_MEMBERSHIPID', 'FECHA_TRANSACCION'], ascending=[False, True, True], ignore_index=True)
)

# Favorite cinema per customer (mode of historical visits)
df_transactions['favorite_cinema'] = (
    df_transactions
    .groupby('CARD_MEMBERSHIPID')['ID_CINE']
    .transform(lambda x: x.mode()[0] if not x.mode().empty else None)
)

df_transactions.head()

---
# 3. ADT Panel Construction (RDD-ready)

We build a daily panel around each email, +/- K days. This creates matched pre/post windows to estimate the email’s near-term effect.

### 3.1: Configure window and filter emails
- We subset emails to a recent time range and convert event dates to pandas datetime.
- The preview above shows the first rows (member, send, date) we will expand into a daily panel.

In [ ]:
# Step 1: Configure window and filter emails
K = 7
print(f"Window size K: {K} days")

# Filter a recent email window and parse dates
_df = (
    df_emails
    .loc[lambda x: (x['EventDate'] >= pd.to_datetime('2024-01-01').date()) &
                  (x['EventDate'] <= pd.to_datetime('2024-07-15').date())]
    .assign(EventDate=lambda x: pd.to_datetime(x['EventDate'], format='%Y-%m-%d', errors='coerce'))
)
print("Emails in window:", _df.shape)

# Preview the filtered emails (first 5)
_df[['CARD_MEMBERSHIPID', 'SendId', 'EventDate']].head()

#### 3.2: Expand emails into daily windows
- For each email date, we create a list of dates from K days before to K days after.
- The preview above shows the list we will explode into one row per day.

In [ ]:
# 3.2 Step 2: Build +/-K day ranges per email
_df_ranges = _df.assign(
    dates_to_expand=lambda x: x['EventDate'].apply(lambda d: pd.date_range(d - pd.Timedelta(days=K), d + pd.Timedelta(days=K), freq='D'))
)
print("Ranges built:", _df_ranges.shape)

# Preview first few range lists
_df_ranges[['CARD_MEMBERSHIPID', 'SendId', 'EventDate', 'dates_to_expand']].head()

#### 3.3: Explode to daily panel
- Exploding the date lists gives us a daily panel per email and customer.
- The head above shows how each email now maps to multiple dates.

In [ ]:
# 3.3 Step 3: Expand to one row per day per email
emails_daily = (
    _df_ranges
    .loc[:, ['CARD_MEMBERSHIPID', 'SendId', 'dates_to_expand']]
    .explode('dates_to_expand')
    .rename(columns={'dates_to_expand': 'EventDate'})
    .reset_index(drop=True)
)
print("Daily rows:", emails_daily.shape)

# Preview first daily rows
emails_daily.head()

#### 3.4: Add relative day index
- We assign a relative day index centered at 0 on the send day; negative is pre, positive is post.
- The head above shows the panel now includes the relative position around each email.

In [ ]:
# 3.4 Step 4: Add relative index from email day and keep [-K, K]
emails_daily['days_from_email'] = emails_daily.groupby(['CARD_MEMBERSHIPID', 'SendId']).cumcount() - K
print("days_from_email min/max:", int(emails_daily['days_from_email'].min()), int(emails_daily['days_from_email'].max()))

emails_daily = emails_daily.query("days_from_email.between(-@K, @K)").reset_index(drop=True)
print("Daily panel after trimming:", emails_daily.shape)

# Preview daily panel with relative day
emails_daily.head()

#### 3.5: Add transactions to panel
- We add daily context features to help control for demand: blockbuster intensity, holidays, releases, and weekly/monthly cycles.
- The head above previews these columns alongside the date.

In [ ]:
# 3.5 Step 5: Enrich with context (blockbuster, calendar, holidays)
emails_daily_ctx = (
    emails_daily
    .merge(ts_movies_on_air[['date', 'blockbuster_score']], how='left', left_on='EventDate', right_on='date')
    .drop(columns=['date'])
    .assign(
        weekday=lambda x: x['EventDate'].dt.day_name(),
        monthday=lambda x: x['EventDate'].dt.day,
        is_holiday=lambda x: x['EventDate'].isin(df_mx_holidays['date']).astype(int),
        is_release_day=lambda x: x['EventDate'].isin(df_releases['ESTRENO']).astype(int),
        sin_weekday=lambda x: np.sin(2 * np.pi * x['EventDate'].dt.weekday / 7),
        cos_weekday=lambda x: np.cos(2 * np.pi * x['EventDate'].dt.weekday / 7),
        sin_monthday=lambda x: np.sin(2 * np.pi * x['EventDate'].dt.day / 30.4),
        cos_monthday=lambda x: np.cos(2 * np.pi * x['EventDate'].dt.day / 30.4)
    )
)
print("Daily panel with context:", emails_daily_ctx.shape)

# Preview context columns
emails_daily_ctx[['EventDate','blockbuster_score','is_holiday','is_release_day','sin_weekday','cos_weekday']].head()

#### 3.6: Flag purchases
- We join daily rows to same-day transactions to flag whether a purchase occurred (has_bought).
- The head above shows purchase flags with select user features used later as controls.

In [ ]:
# 3.6 Step 6: Merge transactions to flag purchase
trx_day = df_transactions.loc[:, ['CARD_MEMBERSHIPID', 'FECHA_TRANSACCION', 'has_children', 'total_visits', 'avg_spend_per_visit', 'favorite_cinema', 'transaction_id']]

emails_daily_ctx_trx = (
    emails_daily_ctx
    .merge(trx_day, how='left', left_on=['CARD_MEMBERSHIPID', 'EventDate'], right_on=['CARD_MEMBERSHIPID', 'FECHA_TRANSACCION'])
    .drop(columns=['FECHA_TRANSACCION'])
    .assign(has_bought=lambda x: np.where(x['transaction_id'].notna(), 1, 0))
)
print("Panel after trx merge:", emails_daily_ctx_trx.shape)
print("Has_bought rate (%):", round(emails_daily_ctx_trx['has_bought'].mean() * 100, 3))

# Preview purchase labels and key user features
emails_daily_ctx_trx[['EventDate','CARD_MEMBERSHIPID','has_bought','has_children','total_visits','favorite_cinema']].head()

#### 3.7: Stabilize user attributes
- We forward/backward fill stable user attributes so each daily row has consistent values.
- The head above shows these stabilized features for inspection.

In [ ]:
# 3.7 Step 7: Stabilize user features across the window (ffill/bfill)
emails_daily_ctx_trx_fb = (
    emails_daily_ctx_trx
    .sort_values(by=['CARD_MEMBERSHIPID', 'EventDate'], ignore_index=True)
    .assign(
        has_children=lambda x: x.groupby('CARD_MEMBERSHIPID')['has_children'].ffill().bfill(),
        avg_spend_per_visit=lambda x: x.groupby('CARD_MEMBERSHIPID')['avg_spend_per_visit'].ffill().bfill(),
        favorite_cinema=lambda x: x.groupby('CARD_MEMBERSHIPID')['favorite_cinema'].ffill().bfill(),
        total_visits=lambda x: x.groupby('CARD_MEMBERSHIPID')['total_visits'].ffill().bfill()
    )
)
print("Panel after ffill/bfill:", emails_daily_ctx_trx_fb.shape)

# Preview stabilized features
emails_daily_ctx_trx_fb[['CARD_MEMBERSHIPID','EventDate','has_children','total_visits','avg_spend_per_visit','favorite_cinema']].head()

#### 3.8: Final panel shape
- Built a balanced pre/post window (K=7 days) around each email.
- Enriched with calendar, holiday, release, and blockbuster context.
- Labeled daily purchase outcomes to enable both ML and RDD estimations.

In [ ]:
# 3.8 Step 8: Define RDD variables and finalize panel
panel_final = emails_daily_ctx_trx_fb.assign(
    is_post=lambda x: x['days_from_email'].gt(0).astype(int),
    r=lambda x: np.where(x['days_from_email'].gt(0), x['days_from_email'], 0),
    w=lambda x: 1 - x['days_from_email'].abs() / (K + 1),  # Triangular kernel weight
)

# Drop the exact email day (0) to avoid exposure mixing
panel_final = panel_final.query("days_from_email != 0").reset_index(drop=True)

print("Final panel shape:", panel_final.shape)
print("Post-period share (%):", round(panel_final['is_post'].mean() * 100, 3))
print("Purchase rate (%):", round(panel_final['has_bought'].mean() * 100, 3))

# Preview finalized panel
panel_final[['CARD_MEMBERSHIPID','EventDate','days_from_email','is_post','w','has_bought']].head()

# Keep downstream variable name
df_emails_expanded = panel_final.copy()

---
# 4. ML Model: Purchase Prediction and Uplift Read

We train CatBoost to predict daily purchase probability in the panel and examine whether the post-email indicator is predictive after controlling for other drivers.

### 4.1: Define features/target and basic checks
We prepare X, y, and sample weights (triangular kernel). We also verify missing values and basic distributions.

In [ ]:
# Select features and target for ML
cols2use = [
    'blockbuster_score',
    'is_holiday',
    'sin_weekday', 'cos_weekday',
    'sin_monthday', 'cos_monthday',
    'has_children', 'total_visits',
    'favorite_cinema', 'SendId',
    'is_post',  # treatment flag
]
cat_cols = ['is_post', 'is_holiday', 'has_children', 'favorite_cinema', 'SendId']

X = df_emails_expanded[cols2use].copy()
X['has_children'] = X['has_children'].fillna(-1).astype(int)
X['is_post'] = X['is_post'].astype(int)
X['is_holiday'] = X['is_holiday'].astype(int)
X['favorite_cinema'] = X['favorite_cinema'].fillna(-1).astype(int)
X['SendId'] = X['SendId'].astype(str)
X[cat_cols] = X[cat_cols].astype('category')

y = df_emails_expanded['has_bought'].copy()
w_weights = df_emails_expanded['w'].copy()  # triangular kernel weights

# Basic checks
print("X shape:", X.shape, " y shape:", y.shape)
print("NaNs per column:\n", X.isna().sum())
print("Class balance (bought=1) %:", round(y.mean()*100,3))

#### 4.2: Split, build pools, and train
- Split into train/validation, stratifying by treatment (is_post) for balance.
- Build CatBoost pools to handle categorical features and sample weights.
- Train with early stopping using the validation set.

In [ ]:
# Step 2: Train/validation split (stratify by treatment)
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
    X, y, w_weights, test_size=0.2, random_state=42, stratify=X['is_post']
)
print(f"Train size: {X_train.shape}, Val size: {X_val.shape}")

# Step 3: Build CatBoost pools
pool_train = Pool(X_train, y_train, weight=w_train, cat_features=cat_cols)
pool_val = Pool(X_val, y_val, weight=w_val, cat_features=cat_cols)

# Step 4: Train CatBoost
model = CatBoostClassifier(
    loss_function="Logloss",
    random_seed=42,
    verbose=100,
    learning_rate=0.1,
)
model.fit(pool_train, eval_set=pool_val, early_stopping_rounds=50)

### 4.3: Evaluate and interpret
- Report precision/recall/F1 for both classes.
- Use SHAP to understand global importance and targeted relationships (including is_post).

In [ ]:
# Step 5: Evaluate classification metrics on validation set
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=['Not Bought', 'Bought'], digits=4))

# Step 6: SHAP interpretation
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_val)

# Global importance summary
shap.summary_plot(shap_values, X_val, plot_type="dot", max_display=20)

# Top features by mean |SHAP|
shap_abs = np.abs(shap_values)
feat_importance = pd.Series(shap_abs.mean(axis=0), index=X_val.columns).sort_values(ascending=False)
print("Top features by mean |SHAP|:\n", feat_importance.head(10))

# Targeted partials (including treatment)
for feat in ['is_post', 'blockbuster_score', 'total_visits', 'is_holiday']:
    shap.dependence_plot(feat, shap_values, X_val, interaction_index=None, show=False)
    plt.title(f'Partial Dependence of {feat}')
    plt.show()

#### Insights: ML Model
- The model captures calendar effects and user history while assessing the post-email flag.
- Inspect SHAP for whether 'is_post' meaningfully shifts purchase probability after controlling for confounders.
- Use caution: model-based association is not causal, but it helps detect patterns that RDD might miss.

---
# 5. RDD: GLM with Cluster-Robust Errors

We estimate the post-email jump using a simple logistic GLM, clustering standard errors by customer to account for within-user correlation.

### 5.1: Fit Linear Model with robust SEs
We classify the day type and build a compact GLM formula with treatment and calendar controls.

In [ ]:
# Step 1a: Day type categorical: weekend, Wednesday, other (baseline)
df_emails_expanded['type_of_day'] = np.where(
    df_emails_expanded['EventDate'].dt.weekday.isin([5, 6]), 'weekend',
    np.where(df_emails_expanded['EventDate'].dt.weekday == 2, 'wednesday', 'other')
)
print(df_emails_expanded['type_of_day'].value_counts(normalize=True).rename('%').mul(100).round(2))

# Step 1b: Simple RDD formula: treatment + calendar controls
formula = "has_bought ~ is_post + C(type_of_day, Treatment('other')) + is_holiday"
print("RDD formula:", formula)

# Step 2: Ensure no missing values in used fields
df_emails_expanded['has_children'] = df_emails_expanded['has_children'].fillna(0).astype(int)

# Step 3: Fit GLM Binomial with cluster-robust SE at customer level
rdd_model = smf.glm(formula, data=df_emails_expanded, family=sm.families.Binomial())
glm_res = rdd_model.fit(cov_type='cluster', cov_kwds={'groups': df_emails_expanded['CARD_MEMBERSHIPID']})

# Step 4: Report
tbl = glm_res.summary()
print("RD jump (is_post) on log-odds:", glm_res.params['is_post'])
print("RD jump (is_post) as odds ratio:", np.exp(glm_res.params['is_post']))

In [ ]:
# look table
print(tbl)

## 5.2 Plot effects

In [ ]:
# Step 5: Plot coefficients with 95% CIs
coef_df = pd.DataFrame({
    'feature': glm_res.params.index,
    'coef': glm_res.params.values,
    'conf_int_lower': glm_res.conf_int()[0],
    'conf_int_upper': glm_res.conf_int()[1],
    'p_value': glm_res.pvalues.values
})
coef_df = coef_df[coef_df['feature'] != 'Intercept']
coef_df['feature'] = coef_df['feature'].str.replace(r"C\(type_of_day, Treatment\('other'\)\)\[T\.", '', regex=True)
coef_df['feature'] = coef_df['feature'].str.replace(r"\]", '', regex=True)
coef_df['color'] = np.where(coef_df['p_value'] < 0.05, 'blue', 'red')

plt.figure(figsize=(10, 6))
for _, row in coef_df.iterrows():
    plt.errorbar(
        row['feature'], row['coef'],
        yerr=[[row['coef'] - row['conf_int_lower']], [row['conf_int_upper'] - row['coef']]],
        fmt='o', color=row['color'], capsize=5, elinewidth=1, markeredgewidth=1
    )
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.xticks(rotation=0)
plt.title('RDD Coefficients with 95% Confidence Intervals')
plt.xlabel('Features')
plt.ylabel('Coefficient')
plt.tight_layout()
plt.show()


In [ ]:
# Step 6: Predictive sanity check
y_pred = glm_res.predict(df_emails_expanded)
print("Share predicted >0.5:", float((y_pred > 0.5).mean()))

#### Insights: RDD GLM
- The coefficient on 'is_post' measures the discontinuity in purchase probability right after emails, controlling for day-type and holidays, with customer-clustered SEs.
- In this run, the estimate is small and statistically insignificant, indicating no clear uplift.
- Caveats: local design (+/- K days), daily aggregation, and potential unobserved confounding (e.g., time-of-day or concurrent promos).

---
# Conclusion and Next Steps

Summary
- No statistically significant uplift detected in purchases immediately after emails in this specification.
- ML reveals calendar and user-history as dominant drivers; the post-email signal is weak once controls are applied.

Next steps
- Increase granularity: incorporate send hour and within-day purchase timing.
- Explore heterogeneous effects (e.g., by visit frequency, family presence, cinema region).
- Bayesian RDD to quantify uncertainty more fully.
- Design an RCT to establish causality and calibrate expected lift.
- Consider geo experiments (treated vs. control cities) to reduce spillovers.